<a href="https://colab.research.google.com/github/DrJHSIM/PyTorch_Practice/blob/main/EarlyStopping_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import torch.optim as optim
import torch.nn.functional as F

import torchvision.models as models
from torchsummary import summary

In [13]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=1),
    ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform = transform)


# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
from torch.utils.data import random_split
generator = torch.Generator().manual_seed(777)
train_set, valid_set = random_split(training_data, [0.8, 0.2], generator = generator)

len(training_data), len(train_set), len(valid_set)

(50000, 40000, 10000)

In [16]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_set, batch_size=batch_size)
valid_dataloader = DataLoader(valid_set, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [17]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [18]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnns = nn.Sequential(
            #conv1
            nn.Conv2d(3, 96 , 11, stride = 4),
            nn.ReLU(),
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            #conv2
            nn.Conv2d(96, 256, 5, stride = 1, padding = 2),
            nn.ReLU(),
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            #conv3
            nn.Conv2d(256, 384, 3, stride = 1, padding = 1),
            nn.ReLU(),
            #conv4
            nn.Conv2d(384, 384, 3, stride = 1, padding = 1),
            nn.ReLU(),
            #conv5
            nn.Conv2d(384, 256, 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
        )

        self.classifier = nn.Sequential(
            #fc1
            nn.Linear(6400, 4096),
            nn.ReLU(),
            nn.Dropout(),
            #fc2
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            #fc3
            nn.Linear(4096, 10)
        )

    def init_weights_bias(self):
        for layer in self.cnns:
            if isinstance(layer, nn.Conv2d):
                nn.init.norm_(layer.weights, mean = 0, std =0.01)
                nn.init.constant_(layer.bias, 0)
    #conv 2, 4, 5는 bias 1로 초기화
        nn.init.constant_(self.cnns[4].bias, 1)
        nn.init.constant_(self.cnns[10].bias, 1)
        nn.init.constant_(self.cnns[12].bias, 1)

    def forward(self, x):
        x = self.cnns(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = AlexNet().to(device)
print(model)

AlexNet(
  (cnns): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2.0)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inpla

In [19]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          34,944
              ReLU-2           [-1, 96, 54, 54]               0
 LocalResponseNorm-3           [-1, 96, 54, 54]               0
         MaxPool2d-4           [-1, 96, 26, 26]               0
            Conv2d-5          [-1, 256, 26, 26]         614,656
              ReLU-6          [-1, 256, 26, 26]               0
 LocalResponseNorm-7          [-1, 256, 26, 26]               0
         MaxPool2d-8          [-1, 256, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]         885,120
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 384, 12, 12]       1,327,488
             ReLU-12          [-1, 384, 12, 12]               0
           Conv2d-13          [-1, 256, 12, 12]         884,992
             ReLU-14          [-1, 256,

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum = 0.9, weight_decay = 0.0005)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [22]:
def valid(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    model.eval()
    valid_loss, Acurracy = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            valid_loss += loss_fn(pred, y).item()
            Acurracy += (pred.argmax(1) == y).type(torch.float).sum().item()
    valid_loss /= num_batches
    Acurracy /= size
    print(f"Valid Error : \n Accuracy : {(Acurracy*100):>0.1f}%, epoch_valid_loss: {valid_loss:>8f} \n")
    return valid_loss # earlystopping시 적용

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 추가 실험

### EarlyStopping 적용

In [24]:
epochs = 30
min_loss = float('inf')  # 검증데이터셋의 최소 loss값
es_con = 5               # 학습 종료 조건
es_counts = 0            # 검증 데이터 loss가 상승하는 epochs의 수

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    valid_loss = valid(valid_dataloader, model, loss_fn)
    if valid_loss > min_loss:
        es_counts += 1
    else:
        min_loss = valid_loss
        es_counts = 0
    if es_counts == es_con:
        print("EarlyStopping")
        break
    print(min_loss, es_con, es_counts)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302305  [   64/40000]
loss: 2.302297  [ 6464/40000]
loss: 2.299439  [12864/40000]
loss: 2.047245  [19264/40000]
loss: 1.955898  [25664/40000]
loss: 1.912456  [32064/40000]
loss: 1.890727  [38464/40000]
Valid Error : 
 Accuracy : 30.3%, epoch_valid_loss: 1.838696 

1.838695923234247 5 0
Epoch 2
-------------------------------
loss: 1.884799  [   64/40000]
loss: 1.896578  [ 6464/40000]
loss: 1.545689  [12864/40000]
loss: 1.398454  [19264/40000]
loss: 1.582658  [25664/40000]
loss: 1.283064  [32064/40000]
loss: 1.613618  [38464/40000]
Valid Error : 
 Accuracy : 46.4%, epoch_valid_loss: 1.460210 

1.4602095169626224 5 0
Epoch 3
-------------------------------
loss: 1.312006  [   64/40000]
loss: 1.543977  [ 6464/40000]
loss: 1.166419  [12864/40000]
loss: 1.124926  [19264/40000]
loss: 1.198185  [25664/40000]
loss: 1.099540  [32064/40000]
loss: 1.410838  [38464/40000]
Valid Error : 
 Accuracy : 57.5%, epoch_valid_loss: 1.200006 

1.20000603130668